In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1, bias=False) #28
        self.norm1 = nn.BatchNorm2d(8)
        self.drop = nn.Dropout2d(0.1)
        self.conv2 = nn.Conv2d(8, 12, 3, padding=1, bias=False) #28
        self.norm2 = nn.BatchNorm2d(12)
        self.drop = nn.Dropout2d(0.1)
        self.pool = nn.MaxPool2d(2, 2) #14

        self.conv3 = nn.Conv2d(12, 16, 3, padding=1, bias=False) #14
        self.norm3 = nn.BatchNorm2d(16)
        self.drop = nn.Dropout2d(0.1)
        self.conv4 = nn.Conv2d(16, 20, 3, padding=1, bias=False) #14
        self.norm4 = nn.BatchNorm2d(20)
        self.drop = nn.Dropout2d(0.1)
        self.pool = nn.MaxPool2d(2,2) #7

        self.conv5 = nn.Conv2d(20, 24, 3, bias=False) #5
        self.norm5 = nn.BatchNorm2d(24)
        self.conv6 = nn.Conv2d(24, 28, 3, bias=False) #3
        self.antman = nn.Conv2d(28, 10 , 1, bias=False)#3
        self.gap = nn.AvgPool2d(3)#1

    def forward(self, x):
      x = F.relu(self.conv1(x))
      x = self.norm1(x)
      x = self.drop(x)
      x = F.relu(self.conv2(x))
      x = self.norm2(x)
      x = self.drop(x)
      x = self.pool(x)

      x = F.relu(self.conv3(x))
      x = self.norm3(x)
      x = self.drop(x)
      x = F.relu(self.conv4(x))
      x = self.norm4(x)
      x = self.drop(x)
      x = self.pool(x)

      x = F.relu(self.conv5(x))
      x = self.norm5(x)
      x = F.relu(self.conv6(x))
      x = self.antman(x)
      x = self.gap(x)
      x = x.view(-1, 10)

      return F.log_softmax(x)

In [28]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         Dropout2d-3            [-1, 8, 28, 28]               0
            Conv2d-4           [-1, 12, 28, 28]             864
       BatchNorm2d-5           [-1, 12, 28, 28]              24
         Dropout2d-6           [-1, 12, 28, 28]               0
         MaxPool2d-7           [-1, 12, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           1,728
       BatchNorm2d-9           [-1, 16, 14, 14]              32
        Dropout2d-10           [-1, 16, 14, 14]               0
           Conv2d-11           [-1, 20, 14, 14]           2,880
      BatchNorm2d-12           [-1, 20, 14, 14]              40
        Dropout2d-13           [-1, 20, 14, 14]               0
        MaxPool2d-14             [-1, 2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [31]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    print('Epoch: ',epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/938 [00:00<?, ?it/s]

Epoch:  1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0479, Accuracy: 9838/10000 (98.38%)

Epoch:  2


  0%|          | 0/938 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0182, Accuracy: 9941/10000 (99.41%)

Epoch:  19


  0%|          | 0/938 [00:00<?, ?it/s]


Test set: Average loss: 0.0188, Accuracy: 9939/10000 (99.39%)

Epoch:  20


loss=0.01460951566696167 batch_id=937: 100%|██████████| 938/938 [00:14<00:00, 63.97it/s]



Test set: Average loss: 0.0161, Accuracy: 9953/10000 (99.53%)

